# Tutorial Part 5: Creating Models with TensorFlow and PyTorch
지금까지 튜토리얼에서는 DeepChem에서 제공하는 표준 모델을 사용해 왔다. 이는 많은 애플리케이션에 적합하지만, 머지않아 사용자가 직접 정의하는 아키텍처를 통해 완전히 새로운 모델을 만들고자 할 것입니다. DeepChem은 TensorFlow(Keras) 및 PyTorch와의 통합을 제공하므로 이러한 프레임워크 중 하나의 모델과 함께 사용할 수 있다.

실제로 DeepChem과 함께 TensorFlow 또는 PyTorch 모델을 사용하기 위해 취할 수 있는 두 가지 다른 접근 방식이 있습니다.  TensorFlow/PyTorch APIs 또는 DeepChem APIs를 사용하여 모델을 교육하고 평가할 것인지 여부에 따라 다릅니다.  전자의 경우, DeepChem의 `Dataset` 클래스는 다른 프레임워크와 함께 사용하기 위해 쉽게 적응하는 방법을 가지고 있다. `make_tf_dataset()`는 데이터 위에 반복되는 `tensorflow.data.Dataset` 개체를 반환합니다. `make_pytorch_dataset()`는 데이터 위에 반복되는 `torch.utils.data.IterableDataset`를 반환합니다.  이를 통해 DeepChem의 datasets, loaders, featurizers, transformers, splitters 등을 사용하고 이를 기존 TensorFlow 또는 PyTorch 코드에 쉽게 통합할 수 있습니다.

그러나 DeepChem은 또한 많은 다른 유용한 특징들을 제공한다.  이러한 기능을 사용할 수 있는 또 다른 방법은 DeepChem `Model` 개체로 모델을 포장하는 것입니다.  어떻게 하는지 봅시다.

## KerasModel

`KerasModel`는 딥켐의 `Model` 클래스의 하위 클래스이다.  그것은 `tensorflow.keras.Model` 주위에 포장지 역할을 한다.  그것을 사용하는 예를 봅시다.  이 예에서는 two dense layers으로 구성된 simple sequential model을 생성한다.

In [1]:
import deepchem as dc
import tensorflow as tf

keras_model = tf.keras.Sequential([
    tf.keras.layers.Dense(1000, activation='relu'),
    tf.keras.layers.Dropout(rate = 0.5),
    tf.keras.layers.Dense(1)
])
model = dc.models.KerasModel(keras_model, dc.models.losses.L2Loss())

이 예에서는 Keras `Sequential` 클래스를 사용했습니다. 우리의 모델은 ReLU activation이 있는 dense layer, regularization을 제공하기 위한 50% dropout 및 final layer을 생성하는 최종 계층으로 구성된다. 우리는 또한 모델을 훈련시킬 때 사용할 loss function를 지정해야 한다. 이 경우 L<sub>2</sub> loss이다. 이제 우리는 다른 DeepChem 모델과 마찬가지로 모델을 교육하고 평가할 수 있습니다. 예를 들어 Delaney 솔루션 데이터 세트를 로드해 보겠습니다. 우리의 모델은 extended-connectivity fingerprints(ECFP)을 기반으로 분자의 용해도를 예측하는 데 어떻게 하는가?

In [2]:
tasks, datasets, transformers = dc.molnet.load_delaney(featurizer='ECFP', splitter='random')
train_dataset, valid_dataset, test_dataset = datasets
model.fit(train_dataset, nb_epoch = 50)
metric = dc.metrics.Metric(dc.metrics.pearson_r2_score)
print('training set score : ', model.evaluate(train_dataset, [metric]))
print('test set score : ', model.evaluate(test_dataset, [metric]))

training set score :  {'pearson_r2_score': 0.9841345965332033}
test set score :  {'pearson_r2_score': 0.7499035876886528}


## TorchModel

`TorchModel`은 `KerasModel`와 똑같이 작동하지만, `torch.nn.Module`를 감싼다는 것만 빼면 됩니다. PyTorch를 사용하여 이전 모델과 마찬가지로 다른 모델을 만들어 동일한 데이터에 대해 교육해 보겠습니다.

In [6]:
!pip install torch

  Using cached torch-1.8.1-cp37-cp37m-win_amd64.whl (190.5 MB)


In [6]:
# deepchem 2.5.0은 TorchModel이 존재하지 않음
!pip install deepchem=2.4.0
dc.__version__

'2.4.0'

In [7]:
import torch

pytorch_model = torch.nn.Sequential(
    torch.nn.Linear(1024, 1000),
    torch.nn.ReLU(),
    torch.nn.Dropout(0.5),
    torch.nn.Linear(1000, 1)
)
model = dc.models.TorchModel(pytorch_model, dc.models.losses.L2Loss())

model.fit(train_dataset, nb_epoch=50)
print('training set score:', model.evaluate(train_dataset, [metric]))
print('test set score:', model.evaluate(test_dataset, [metric]))

training set score: {'pearson_r2_score': 0.9839648176699037}
test set score: {'pearson_r2_score': 0.7385308044943584}


## Computing Losses

이제 좀 더 발전된 예를 보겠습니다. 위의 모델에서 손실은 모델의 출력에서 직접 계산되었다. 종종 그것은 괜찮지만 항상 그렇지는 않다. 확률 분포를 출력하는 분류 모형을 고려합니다. 확률에서 손실을 계산할 수는 있지만 로짓에서 손실을 계산하는 것은 수치적으로 더 안정적입니다.

이를 위해 확률과 로짓 모두 여러 출력을 반환하는 모델을 만듭니다. `KerasModel` and `TorchModel`를 사용하여 "output types" 목록을 지정할 수 있습니다.  특정 출력에 유형이 `'prediction'`인 경우, 이는 `predict()`를 호출할 때 반환되어야 하는 정상 출력임을 의미합니다.  유형 `'loss'`가 있는 경우 정상 출력 대신 손실 함수로 전달해야 합니다.

Sequential models은 다중 출력을 허용하지 않으므로 대신 하위 분류 스타일 모델을 사용한다.

In [8]:
class ClassificationModel(tf.keras.Model):
    
    def __init__(self):
        super(ClassificationModel, self).__init__()
        self.dense1 = tf.keras.layers.Dense(1000, activation='relu')
        self.dense2 = tf.keras.layers.Dense(1)
    
    def call(self, inputs, training=False):
        y = self.dense1(inputs)
        if training:
            y = tf.nn.dropout(y, 0.5)
        logits = self.dense2(y)
        output = tf.nn.sigmoid(logits)
        return output, logits

keras_model= ClassificationModel()
output_types = ['prediction', 'loss']
model = dc.models.KerasModel(keras_model, dc.models.losses.SigmoidCrossEntropy(), output_types = output_types)

우리는 BACE 데이터 세트에 대한 우리의 모델을 훈련시킬 수 있다. 이것은 분자가 BACE-1 효소를 억제할 것인지를 예측하기 위한 이진 분류 작업이다.

In [13]:
tasks, datasets, transformers = dc.molnet.load_bace_classification(feturizer='ECFP', splitter='scaffold')
train_dataset, valid_dataset, test_dataset = datasets
model.fit(train_dataset, nb_epoch=100)
metric = dc.metrics.Metric(dc.metrics.roc_auc_score)
print('training set score:', model.evaluate(train_dataset, [metric]))
print('test set score:', model.evaluate(test_dataset, [metric]))

training set score: {'roc_auc_score': 0.9996116504854369}
test set score: {'roc_auc_score': 0.7600996376811594}


## Other Features
`KerasModel` and `TorchModel`은 다른 특징이 많다.  여기 더 중요한 것들 중 몇 가지가 있다.

- 교육 중 체크포인트 자동 저장
- 콘솔에서 [TensorBoard](https://www.tensorflow.org/tensorboard), 또는 [Weights & Biased](https://docs.wandb.com/)로 진행 상황을 기록합니다.
- `f(outputs, labels, weights)` 형식의 함수로 정의하는 사용자 정의 손실 함수
- `ValidationCallback` 클래스 사용을 조기에 중지합니다.
- 사전 학습된 모델에서 매개 변수 로드
- 모델 출력의 불확실성 추정
- 솔리시티 맵핑을 통해 중요한 기능 식별

`KerasModel` or `TorchModel` 등으로 자신의 모델을 포장하면 이 모든 기능에 즉시 액세스할 수 있다.  자세한 내용은 API 설명서를 참조하십시오.